# Data Processing and Feature Engineering

The train data has all the transactions before October 15, 2016, plus some of the transactions after October 15, 2016

The test data:
* **public leaderboard** has the rest of the transactions between October 15 and December 31, 2016
* **private leaderboard** uses data between October 17 and December 15, 2017.

You are asked to predict 6 time points for all properties: **October 2016 (201610), November 2016 (201611), December 2016 (201612), October 2017 (201710), November 2017 (201711), and December 2017 (201712)**. Not all the properties are sold in each time period. If a property was not sold in a certain time period, that particular row will be ignored when calculating your score.

Note: In this code, assume pass public leaderboard and go to private leaderboard. So we need predict 3 time points: **October 2017 (201710), November 2017 (201711), and December 2017 (201712)**

In [1]:
# importing sys
import sys
  
# adding srcpy to the system path
sys.path.insert(0, "/Users/charles/Desktop/iFixerup/zr1/src/srcpy/")

import data_proc, feature_proc

# Auto reload: watch a directory for changed files and restarts a process when the change is detected
%load_ext autoreload
%autoreload 2

In [3]:
import random
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None
pd.options.display.float_format

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
mpl.style.use("ggplot")
pylab.rcParams["figure.figsize"] = 8 , 6

import seaborn as sns
sns.set_style("white")

# Data loading and Preprocessing

In [4]:
%%time
# Load in properties data
prop_2016 = data_proc.load_properties_data("/Users/charles/Desktop/iFixerup/zr1/data/csv/properties_2016.csv")
prop_2017 = data_proc.load_properties_data("/Users/charles/Desktop/iFixerup/zr1/data/csv/properties_2017.csv")

# Rename columns 
data_proc.rename_columns(prop_2016)
data_proc.rename_columns(prop_2017)

assert len(prop_2016) == len(prop_2017)

print("Number of properties: {}".format(len(prop_2016)))
print("Number of property features: {}".format(len(prop_2016.columns)-1))

Number of properties: 2985217
Number of property features: 57
CPU times: user 15.4 s, sys: 3.24 s, total: 18.6 s
Wall time: 20.9 s


In [5]:
%%time
# Get land use & zoning desc
df_landuse_codes = feature_proc.get_landuse_code_df(prop_2016, prop_2017)
df_zoning_codes = feature_proc.get_zoning_desc_code_df(prop_2016, prop_2017)

# Process columns 
prop_2016 = feature_proc.process_columns(prop_2016, df_landuse_codes, df_zoning_codes)
prop_2017 = feature_proc.process_columns(prop_2017, df_landuse_codes, df_zoning_codes)

# Retype columns 
data_proc.retype_columns(prop_2016)
data_proc.retype_columns(prop_2017)

prop_2017.head()

CPU times: user 21.2 s, sys: 37.2 s, total: 58.4 s
Wall time: 1min 10s


,parcel_id,cooling_id,architecture_style_id,basement_sqft,bathroom_cnt,bedroom_cnt,framing_id,quality_id,bathroom_cnt_calc,deck_id,floor1_sqft,finished_area_sqft_calc,finished_area_sqft,perimeter_area,total_area,floor1_sqft_unk,base_total_area,fips,fireplace_cnt,bathroom_full_cnt,garage_cnt,garage_sqft,spa_flag,heating_id,latitude,longitude,lot_sqft,pool_cnt,pool_total_size,pool_or_sht,pool_w_sht,pool_no_sht,landuse_type_id,census_raw,city_id,country_id,neighborhood_id,region_zip,room_cnt,story_id,bathroom_small_cnt,construction_id,unit_cnt,patio_sqft,storage_sqft,year_built,story_cnt,fireplace_flag,tax_structure,tax_parcel,tax_year,tax_land,tax_property,tax_overdue_flag,tax_overdue_year,census_2,country_landuse_code_id,zoning_description_id
0,10754147,-1,-1,NaN,0.0,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34144440.0,-118654080.0,85768.0,NaN,NaN,NaN,NaN,NaN,238,60378004.0,37688.0,0,NaN,96337.0,0.0,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN,9.0,NaN
1,10759547,-1,-1,NaN,0.0,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34140432.0,-118625360.0,4083.0,NaN,NaN,NaN,NaN,NaN,230,60378000.0,37688.0,0,NaN,96337.0,0.0,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN,49.0,29.0
2,10843547,-1,-1,NaN,0.0,0.0,4,NaN,NaN,NaN,NaN,73026.0,NaN,NaN,73026.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,33989360.0,-118394632.0,63085.0,NaN,NaN,NaN,NaN,NaN,16,60377032.0,51617.0,0,NaN,96095.0,0.0,NaN,NaN,-1,2.0,NaN,NaN,1959.0,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.369141,NaN,NaN,NaN,1.0,36.0
3,10859147,-1,-1,NaN,0.0,0.0,2,6.0,NaN,NaN,NaN,5068.0,NaN,NaN,5068.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34148864.0,-118437208.0,7521.0,NaN,NaN,NaN,NaN,NaN,16,60371412.0,12447.0,0,27080.0,96424.0,0.0,NaN,NaN,-1,NaN,NaN,NaN,1948.0,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.570312,NaN,NaN,NaN,1.0,36.0
4,10879947,-1,-1,NaN,0.0,0.0,3,NaN,NaN,NaN,NaN,1776.0,NaN,NaN,1776.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34194168.0,-118385816.0,8512.0,NaN,NaN,NaN,NaN,NaN,0,60371232.0,12447.0,0,46795.0,96450.0,0.0,NaN,NaN,-1,1.0,NaN,NaN,1947.0,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.169922,NaN,NaN,NaN,30.0,NaN


In [6]:
%%time
# Write current DataFrames to hdf5
prop_2016.to_hdf('/Users/charles/Desktop/iFixerup/zr1/data/hdf5/prop.h5', key='prop_2016', format='table', mode='w')
prop_2017.to_hdf('/Users/charles/Desktop/iFixerup/zr1/data/hdf5/prop.h5', key='prop_2017', format='table', mode='a')

CPU times: user 3.47 s, sys: 1.65 s, total: 5.12 s
Wall time: 6.56 s


In [7]:
%%time
# Read DataFrames from hdf5
prop_2016 = pd.read_hdf('/Users/charles/Desktop/iFixerup/zr1/data/hdf5/prop.h5', 'prop_2016')
prop_2017 = pd.read_hdf('/Users/charles/Desktop/iFixerup/zr1/data/hdf5/prop.h5', 'prop_2017')

CPU times: user 2.79 s, sys: 1.34 s, total: 4.13 s
Wall time: 4.67 s


In [8]:
# Load in training data (with log_error labels)
train_2016 = data_proc.load_training_data("/Users/charles/Desktop/iFixerup/zr1/data/csv/train_2016P.csv")
train_2017 = data_proc.load_training_data("/Users/charles/Desktop/iFixerup/zr1/data/csv/train_2017.csv")

print("Number of 2016 transaction records: {}".format(len(train_2016)))
print("Number of 2017 transaction records: {}".format(len(train_2017)))
print("\n", train_2016.tail())
print("\n", train_2017.tail())

Number of 2016 transaction records: 90275
Number of 2017 transaction records: 77613

        parcel_id  log_error transaction_date
90270   10774160    -0.0356       2016-12-30
90271   12046695     0.0070       2016-12-30
90272   12995401    -0.2679       2016-12-30
90273   11402105     0.0602       2016-12-30
90274   12566293     0.4207       2016-12-30

        parcel_id  log_error transaction_date
77608   10833991  -0.002245       2017-09-20
77609   11000655   0.020615       2017-09-20
77610   17239384   0.013209       2017-09-21
77611   12773139   0.037129       2017-09-21
77612   12826780   0.007204       2017-09-25


# Feature Engineering

In [9]:
# Basic feature engineering + Drop duplicate columns
prop_2016, prop_2017 = feature_proc.feature_engineering(prop_2016, prop_2017)

prop_2017.head()

,parcel_id,cooling_id,architecture_style_id,basement_sqft,bathroom_cnt,bedroom_cnt,framing_id,quality_id,deck_id,floor1_sqft,finished_area_sqft_calc,perimeter_area,floor1_sqft_unk,base_total_area,fips,fireplace_cnt,bathroom_full_cnt,garage_cnt,garage_sqft,spa_flag,heating_id,latitude,longitude,lot_sqft,pool_cnt,pool_total_size,pool_or_sht,pool_w_sht,pool_no_sht,landuse_type_id,census_raw,city_id,country_id,neighborhood_id,region_zip,room_cnt,story_id,bathroom_small_cnt,construction_id,unit_cnt,patio_sqft,storage_sqft,year_built,story_cnt,fireplace_flag,tax_structure,tax_parcel,tax_year,tax_land,tax_property,tax_overdue_flag,tax_overdue_year,census_2,country_landuse_code_id,zoning_description_id,avg_garage_size,property_tax_per_sqft,location_sum,location_minus,location_sum05,location_minus05,missing_finished_area,missing_total_area,missing_bathroom_cnt_calc,derived_room_cnt,avg_area_per_room,derived_avg_area_per_room
0,10754147,-1,-1,NaN,0.0,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34144440.0,-118654080.0,85768.0,NaN,NaN,NaN,NaN,NaN,238,60378004.0,37688.0,0,NaN,96337.0,0.0,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,-84509640.0,152798528.0,-25182600.0,93471480.0,1.0,1.0,1.0,0.0,NaN,NaN
1,10759547,-1,-1,NaN,0.0,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34140432.0,-118625360.0,4083.0,NaN,NaN,NaN,NaN,NaN,230,60378000.0,37688.0,0,NaN,96337.0,0.0,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN,49.0,29.0,NaN,NaN,-84484928.0,152765792.0,-25172248.0,93453112.0,1.0,1.0,1.0,0.0,NaN,NaN
2,10843547,-1,-1,NaN,0.0,0.0,4,NaN,NaN,NaN,73026.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,33989360.0,-118394632.0,63085.0,NaN,NaN,NaN,NaN,NaN,16,60377032.0,51617.0,0,NaN,96095.0,0.0,NaN,NaN,-1,2.0,NaN,NaN,1959.0,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.369141,NaN,NaN,NaN,1.0,36.0,NaN,0.284835,-84405272.0,152384000.0,-25207956.0,93186676.0,1.0,0.0,1.0,0.0,NaN,NaN
3,10859147,-1,-1,NaN,0.0,0.0,2,6.0,NaN,NaN,5068.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34148864.0,-118437208.0,7521.0,NaN,NaN,NaN,NaN,NaN,16,60371412.0,12447.0,0,27080.0,96424.0,0.0,NaN,NaN,-1,NaN,NaN,NaN,1948.0,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.570312,NaN,NaN,NaN,1.0,36.0,NaN,2.872449,-84288344.0,152586080.0,-25069740.0,93367468.0,1.0,0.0,1.0,0.0,NaN,NaN
4,10879947,-1,-1,NaN,0.0,0.0,3,NaN,NaN,NaN,1776.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34194168.0,-118385816.0,8512.0,NaN,NaN,NaN,NaN,NaN,0,60371232.0,12447.0,0,46795.0,96450.0,0.0,NaN,NaN,-1,1.0,NaN,NaN,1947.0,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.169922,NaN,NaN,NaN,30.0,NaN,NaN,3.223632,-84191648.0,152579984.0,-24998740.0,93387076.0,1.0,0.0,1.0,0.0,NaN,NaN


In [10]:
# Compute region-based aggregate features
group_col = 'region_zip'
agg_cols = ['lot_sqft', 'year_built', 'finished_area_sqft_calc',
            'tax_structure', 'tax_land', 'tax_property', 'property_tax_per_sqft']

prop_2016 = feature_proc.region_aggregate_features(prop_2016, group_col, agg_cols)
prop_2017 = feature_proc.region_aggregate_features(prop_2017, group_col, agg_cols)

prop_2017.head()

,parcel_id,cooling_id,architecture_style_id,basement_sqft,bathroom_cnt,bedroom_cnt,framing_id,quality_id,deck_id,floor1_sqft,finished_area_sqft_calc,perimeter_area,floor1_sqft_unk,base_total_area,fips,fireplace_cnt,bathroom_full_cnt,garage_cnt,garage_sqft,spa_flag,heating_id,latitude,longitude,lot_sqft,pool_cnt,pool_total_size,pool_or_sht,pool_w_sht,pool_no_sht,landuse_type_id,census_raw,city_id,country_id,neighborhood_id,region_zip,room_cnt,story_id,bathroom_small_cnt,construction_id,unit_cnt,patio_sqft,storage_sqft,year_built,story_cnt,fireplace_flag,tax_structure,tax_parcel,tax_year,tax_land,tax_property,tax_overdue_flag,tax_overdue_year,census_2,country_landuse_code_id,zoning_description_id,avg_garage_size,property_tax_per_sqft,location_sum,location_minus,location_sum05,location_minus05,missing_finished_area,missing_total_area,missing_bathroom_cnt_calc,derived_room_cnt,avg_area_per_room,derived_avg_area_per_room,region_zip-groupcnt,region_zip-lot_sqft-diff,region_zip-lot_sqft-percent,region_zip-year_built-diff,region_zip-finished_area_sqft_calc-diff,region_zip-finished_area_sqft_calc-percent,region_zip-tax_structure-diff,region_zip-tax_structure-percent,region_zip-tax_land-diff,region_zip-tax_land-percent,region_zip-tax_property-diff,region_zip-tax_property-percent,region_zip-property_tax_per_sqft-diff,region_zip-property_tax_per_sqft-percent
0,10754147,-1,-1,NaN,0.0,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34144440.0,-118654080.0,85768.0,NaN,NaN,NaN,NaN,NaN,238,60378004.0,37688.0,0,NaN,96337.0,0.0,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,-84509640.0,152798528.0,-25182600.0,93471480.0,1.0,1.0,1.0,0.0,NaN,NaN,8496.0,25030.929688,0.412119,NaN,NaN,NaN,NaN,NaN,-551431.062500,-0.999984,NaN,NaN,NaN,NaN
1,10759547,-1,-1,NaN,0.0,0.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34140432.0,-118625360.0,4083.0,NaN,NaN,NaN,NaN,NaN,230,60378000.0,37688.0,0,NaN,96337.0,0.0,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN,49.0,29.0,NaN,NaN,-84484928.0,152765792.0,-25172248.0,93453112.0,1.0,1.0,1.0,0.0,NaN,NaN,8496.0,-56654.070312,-0.932776,NaN,NaN,NaN,NaN,NaN,-523924.062500,-0.950102,NaN,NaN,NaN,NaN
2,10843547,-1,-1,NaN,0.0,0.0,4,NaN,NaN,NaN,73026.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,33989360.0,-118394632.0,63085.0,NaN,NaN,NaN,NaN,NaN,16,60377032.0,51617.0,0,NaN,96095.0,0.0,NaN,NaN,-1,2.0,NaN,NaN,1959.0,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.369141,NaN,NaN,NaN,1.0,36.0,NaN,0.284835,-84405272.0,152384000.0,-25207956.0,93186676.0,1.0,0.0,1.0,0.0,NaN,NaN,9439.0,-19054.117188,-0.231974,-1.895996,71617.218750,50.836166,521561.34375,3.749040,529789.687500,2.167083,15996.112305,3.329571,-3.279059,-0.920078
3,10859147,-1,-1,NaN,0.0,0.0,2,6.0,NaN,NaN,5068.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34148864.0,-118437208.0,7521.0,NaN,NaN,NaN,NaN,NaN,16,60371412.0,12447.0,0,27080.0,96424.0,0.0,NaN,NaN,-1,NaN,NaN,NaN,1948.0,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.570312,NaN,NaN,NaN,1.0,36.0,NaN,2.872449,-84288344.0,152586080.0,-25069740.0,93367468.0,1.0,0.0,1.0,0.0,NaN,NaN,7912.0,-8604.140625,-0.533585,-13.133911,3026.638672,1.482657,327684.28125,1.298404,228256.031250,0.623378,7026.920410,0.933109,-0.825856,-0.223307
4,10879947,-1,-1,NaN,0.0,0.0,3,NaN,NaN,NaN,1776.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,-1,34194168.0,-118385816.0,8512.0,NaN,NaN,NaN,NaN,NaN,0,60371232.0,12447.0,0,46795.0,96450.0,0.0,NaN,NaN,-1,1.0,NaN,NaN,1947.0,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.169922,NaN,NaN,NaN,30.0,NaN,NaN,3.223632,-84191648.0,152579984.0,-24998740.0,93387076.0,1.0,0.0,1.0,0.0,NaN,NaN,7726.0,-10369.894531,-0.549198,-10.163208,205.602173,0.130924,63124.03125,0.472390,87361.265625,0.560049,2136.794189,0.595477,0.814859,0.338288


In [11]:
%%time
# Write feature DataFrames to hdf5
prop_2016.to_hdf('/Users/charles/Desktop/iFixerup/zr1/data/hdf5/features.h5', key='features_2016', format='table', mode='w')
prop_2017.to_hdf('/Users/charles/Desktop/iFixerup/zr1/data/hdf5/features.h5', key='features_2017', format='table', mode='a')

CPU times: user 3.57 s, sys: 1.23 s, total: 4.79 s
Wall time: 6.24 s


# Training data preparation

In [18]:
# Join the training data with the property table
train_2016_merged = train_2016.merge(how='left', right=prop_2016, on='parcel_id')

train_2017_merged = train_2017.merge(how='left', right=prop_2017, on='parcel_id')

train = pd.concat([train_2016_merged, train_2017_merged], axis=0, ignore_index=True)

print("Combined training set size: {}".format(len(train)))

# Add datetime features to training data
data_proc.simple_datetime_features(train)

train.head()

Combined training set size: 167888


,parcel_id,log_error,cooling_id,architecture_style_id,basement_sqft,bathroom_cnt,bedroom_cnt,framing_id,quality_id,deck_id,floor1_sqft,finished_area_sqft_calc,perimeter_area,floor1_sqft_unk,base_total_area,fips,fireplace_cnt,bathroom_full_cnt,garage_cnt,garage_sqft,spa_flag,heating_id,latitude,longitude,lot_sqft,pool_cnt,pool_total_size,pool_or_sht,pool_w_sht,pool_no_sht,landuse_type_id,census_raw,city_id,country_id,neighborhood_id,region_zip,room_cnt,story_id,bathroom_small_cnt,construction_id,unit_cnt,patio_sqft,storage_sqft,year_built,story_cnt,fireplace_flag,tax_structure,tax_parcel,tax_year,tax_land,tax_property,tax_overdue_flag,tax_overdue_year,census_2,country_landuse_code_id,zoning_description_id,avg_garage_size,property_tax_per_sqft,location_sum,location_minus,location_sum05,location_minus05,missing_finished_area,missing_total_area,missing_bathroom_cnt_calc,derived_room_cnt,avg_area_per_room,derived_avg_area_per_room,region_zip-groupcnt,region_zip-lot_sqft-diff,region_zip-lot_sqft-percent,region_zip-year_built-diff,region_zip-finished_area_sqft_calc-diff,region_zip-finished_area_sqft_calc-percent,region_zip-tax_structure-diff,region_zip-tax_structure-percent,region_zip-tax_land-diff,region_zip-tax_land-percent,region_zip-tax_property-diff,region_zip-tax_property-percent,region_zip-property_tax_per_sqft-diff,region_zip-property_tax_per_sqft-percent,year,month,quarter
0,11016594,0.0276,0,-1,NaN,2.0,3.0,-1,4.0,NaN,NaN,1684.0,NaN,NaN,NaN,0,NaN,2.0,NaN,NaN,NaN,1,34280992.0,-118488536.0,7528.0,NaN,NaN,NaN,NaN,NaN,230,60371068.0,12447.0,0,31817.0,96370.0,0.0,NaN,NaN,-1,1.0,NaN,NaN,1959.0,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.879883,NaN,NaN,6.037107e+13,41.0,11.0,NaN,3.999929,-84207544.0,152769536.0,-24963276.0,93525260.0,0.0,1.0,0.0,5.0,NaN,336.799988,14719.0,-13398.968750,-0.640273,-3.998413,-247.725464,-0.128241,-50475.015625,-0.291377,51026.421875,0.273762,2047.035645,0.436576,1.521634,0.613984,0,1,1
1,14366692,-0.1684,-1,-1,NaN,3.5,4.0,-1,NaN,NaN,NaN,2263.0,NaN,NaN,NaN,22,NaN,3.0,2.0,468.0,NaN,-1,33668120.0,-117677552.0,3643.0,NaN,NaN,NaN,NaN,NaN,230,60590524.0,32380.0,1,NaN,96962.0,0.0,NaN,1.0,-1,NaN,NaN,NaN,2014.0,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.019531,NaN,NaN,NaN,15.0,NaN,234.0,4.486531,-84009432.0,151345664.0,-25170656.0,92506896.0,0.0,1.0,0.0,7.5,NaN,301.733337,17682.0,-2715.032715,-0.427024,35.535156,526.538208,0.303225,213678.171875,1.609267,16302.671875,0.073182,6339.847656,1.662618,2.160548,0.928875,0,1,1
2,12098116,-0.0040,0,-1,NaN,3.0,2.0,-1,4.0,NaN,NaN,2217.0,NaN,NaN,NaN,0,NaN,3.0,NaN,NaN,NaN,1,34136312.0,-118175032.0,11423.0,NaN,NaN,NaN,NaN,NaN,230,60374640.0,47019.0,0,275411.0,96293.0,0.0,NaN,NaN,-1,1.0,NaN,NaN,1940.0,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.480469,NaN,NaN,6.037464e+13,41.0,41.0,NaN,5.180190,-84038720.0,152311344.0,-24951204.0,93223828.0,0.0,1.0,0.0,5.0,NaN,443.399994,4422.0,-14927.021484,-0.566490,-12.917847,-173.867432,-0.072721,-236757.281250,-0.792490,-427605.093750,-0.880721,1845.573242,0.191471,1.178391,0.294465,0,1,1
3,12643413,0.0218,0,-1,NaN,2.0,2.0,-1,4.0,NaN,NaN,839.0,NaN,NaN,NaN,0,NaN,2.0,NaN,NaN,NaN,1,33755800.0,-118309000.0,70859.0,NaN,NaN,NaN,NaN,NaN,235,60372964.0,12447.0,0,54300.0,96222.0,0.0,NaN,NaN,-1,1.0,NaN,NaN,1987.0,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.739990,NaN,NaN,6.037296e+13,32.0,60.0,NaN,3.633778,-84553200.0,152064800.0,-25398700.0,92910300.0,0.0,1.0,0.0,4.0,NaN,209.750000,7293.0,-43346.804688,-0.379550,21.690186,-782.150757,-0.482466,30903.765625,0.219777,-129440.812500,-0.638259,-1337.844971,-0.304986,0.830251,0.296145,0,1,1
4,14432541,-0.0050,-1,-1,NaN,2.5,4.0,-1,NaN,NaN,NaN,2283.0,NaN,NaN,NaN,22,NaN,2.0,2.0,598.0,NaN,-1,33485644.0,-117700232.0,6000.0,1.0,NaN,NaN,NaN,1.0,230,60590424.0,17686.0,1,NaN,96961.0,8.0,NaN,1.0,-1,NaN,NaN,NaN,1981.0,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.959961,NaN,NaN,6.059042e+13,4.0,NaN,299.0,2.404275,-84214592.0,151185872.0,-25364472.0,92335760.0,0.0,1.0,0.0,6.5,285.375,351.230774,987

In [19]:
%%time
# Write training DataFrame to hdf5
train.to_hdf('/Users/charles/Desktop/iFixerup/zr1/data/hdf5/train.h5', key='train', format='table', mode='w')

CPU times: user 255 ms, sys: 49.9 ms, total: 305 ms
Wall time: 350 ms
